# 🌌 T-FAN Topology Screensaver - Python Version

**High-performance topology visualization using Ripser, Persim, and Matplotlib**

This notebook computes real topological features from point clouds and visualizes them.

## 🎨 Four Visualization Modes

1. **Barcode Nebula** 🌠 - Persistence barcodes from real topology
2. **Landscape Waterfall** 🌊 - Actual persistence landscapes
3. **Poincaré Orbits** 🪐 - Hyperbolic embeddings
4. **Pareto Galaxy** ⭐ - Multi-objective configurations

## 🚀 Quick Start

1. Run setup cell (installs dependencies)
2. Choose visualization mode
3. Generate and display

**Note:** Computes real topology using Ripser - may take a few seconds

In [ ]:
# Install dependencies
!pip install -q ripser persim numpy scipy matplotlib scikit-learn

print("✅ Dependencies installed!")
print("\n📦 Installed:")
print("  - ripser (fast persistence computation)")
print("  - persim (persistence landscapes)")
print("  - numpy, scipy, matplotlib")
print("\n🎨 Ready to visualize topology!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from ripser import ripser
from persim import PersistenceLandscape
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#020306'
plt.rcParams['axes.facecolor'] = '#020306'

print("✅ Imports complete!")

## 🌠 Barcode Nebula

Compute **real** persistence diagrams from point clouds and visualize as barcodes.

In [ ]:
#@title Generate Barcode Nebula

n_points = 800 #@param {type:"slider", min:200, max:2000, step:100}
n_circles = 3 #@param {type:"slider", min:1, max:5, step:1}
noise_level = 0.06 #@param {type:"slider", min:0.0, max:0.2, step:0.01}

# Generate point cloud with circles (creates H1 features)
def circle_clusters(n=800, k=3, noise=0.06, seed=42):
    np.random.seed(seed)
    pts = []
    for i in range(k):
        r = 1.0 + 0.2 * np.sin(i)
        th = np.random.rand(n//k) * 2 * np.pi
        cx = 2.5 * np.cos(i * 2 * np.pi / k)
        cy = 2.5 * np.sin(i * 2 * np.pi / k)
        x = cx + r * np.cos(th) + noise * np.random.randn(len(th))
        y = cy + r * np.sin(th) + noise * np.random.randn(len(th))
        pts.append(np.stack([x, y], 1))
    return np.concatenate(pts, 0)

print("Generating point cloud...")
P = circle_clusters(n=n_points, k=n_circles, noise=noise_level)

print(f"Computing persistence diagram (n={len(P)} points)...")
result = ripser(P, maxdim=1)
dgms = result['dgms']

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Point cloud
ax1.scatter(P[:, 0], P[:, 1], c='#667eea', s=1, alpha=0.6)
ax1.set_title(f'Point Cloud ({len(P)} points, {n_circles} circles)', fontsize=14, color='white')
ax1.set_xlabel('x', color='white')
ax1.set_ylabel('y', color='white')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.2)

# Barcodes (H1 features)
dgm1 = dgms[1] if len(dgms) > 1 else np.empty((0, 2))
if dgm1.size > 0:
    # Filter infinite bars
    finite_dgm = dgm1[np.isfinite(dgm1[:, 1])]
    
    if len(finite_dgm) > 0:
        # Sort by persistence
        persistence = finite_dgm[:, 1] - finite_dgm[:, 0]
        order = np.argsort(-persistence)
        
        # Plot top barcodes
        for i, idx in enumerate(order[:min(100, len(order))]):
            birth, death = finite_dgm[idx]
            pers = death - birth
            color = cm.viridis(pers / persistence.max())
            ax2.plot([birth, death], [i, i], color=color, linewidth=2, alpha=0.8)
        
        ax2.set_title(f'Persistence Barcodes (H₁ cycles, n={len(finite_dgm)})', fontsize=14, color='white')
        ax2.set_xlabel('Filtration parameter', color='white')
        ax2.set_ylabel('Feature index', color='white')
        ax2.grid(True, alpha=0.2)
    else:
        ax2.text(0.5, 0.5, 'No finite H₁ features', ha='center', va='center', color='white', fontsize=14)
else:
    ax2.text(0.5, 0.5, 'No H₁ features found', ha='center', va='center', color='white', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\n✅ Computed {len(dgm1)} H₁ features (loops/holes)")
if len(dgm1[np.isfinite(dgm1[:, 1])]) > 0:
    max_pers = persistence.max()
    print(f"📊 Max persistence: {max_pers:.3f}")
    print(f"📊 Avg persistence: {persistence.mean():.3f}")

## 🌊 Landscape Waterfall

Compute **actual** persistence landscapes from topology.

In [ ]:
#@title Generate Landscape Waterfall

n_points_land = 600 #@param {type:"slider", min:200, max:1500, step:100}
n_layers = 5 #@param {type:"slider", min:3, max:10, step:1}

# Generate Swiss roll
def swiss_roll(n=600, seed=42):
    np.random.seed(seed)
    theta = np.linspace(1.5*np.pi, 4.5*np.pi, n)
    z = np.linspace(-2, 2, n)
    r = theta + 0.1*np.sin(0.3*theta)
    x = r*np.cos(theta) + 0.05*np.random.randn(n)
    y = r*np.sin(theta) + 0.05*np.random.randn(n)
    return np.stack([x, y], 1)

print("Generating Swiss roll point cloud...")
P = swiss_roll(n=n_points_land)

print(f"Computing persistence (n={len(P)} points)...")
result = ripser(P, maxdim=0)
dgms = result['dgms']

# Compute persistence landscape
dgm0 = dgms[0]
finite_dgm = dgm0[np.isfinite(dgm0[:, 1])]

if len(finite_dgm) > 0:
    print("Computing persistence landscapes...")
    pl = PersistenceLandscape(dgms=[finite_dgm], homology_degree=0)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Point cloud
    ax1.scatter(P[:, 0], P[:, 1], c='#667eea', s=1, alpha=0.6)
    ax1.set_title(f'Swiss Roll ({len(P)} points)', fontsize=14, color='white')
    ax1.set_xlabel('x', color='white')
    ax1.set_ylabel('y', color='white')
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.2)
    
    # Landscapes
    x_range = (0, 3.0)
    xs = np.linspace(x_range[0], x_range[1], 400)
    
    for k in range(1, n_layers + 1):
        ys = pl(xs, k)
        if ys is not None and not np.all(ys == 0):
            color = cm.viridis(k / (n_layers + 1))
            ax2.plot(xs, ys + 0.1*k, color=color, linewidth=2, label=f'λ_{k}', alpha=0.8)
            ax2.fill_between(xs, 0.1*k, ys + 0.1*k, color=color, alpha=0.2)
    
    ax2.set_title(f'Persistence Landscapes (H₀, k={n_layers} layers)', fontsize=14, color='white')
    ax2.set_xlabel('Parameter t', color='white')
    ax2.set_ylabel('λₖ(t)', color='white')
    ax2.legend(loc='upper right', fontsize=10)
    ax2.grid(True, alpha=0.2)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Computed {n_layers} landscape layers")
    print(f"📊 H₀ features: {len(finite_dgm)}")
else:
    print("⚠️  No finite H₀ features found")

## 🪐 Poincaré Orbits

Visualize hyperbolic embeddings on the Poincaré disk.

In [ ]:
#@title Generate Poincaré Orbits

n_points_poincare = 600 #@param {type:"slider", min:200, max:2000, step:100}
n_levels = 6 #@param {type:"slider", min:3, max:10, step:1}

# Generate hierarchical points
np.random.seed(42)
levels = np.random.choice(n_levels, size=n_points_poincare, 
                          p=np.linspace(1, 3, n_levels)/np.linspace(1, 3, n_levels).sum())
r = 0.85 * (levels / levels.max())**0.9 + 0.03 * np.random.rand(n_points_poincare)
theta = np.random.rand(n_points_poincare) * 2 * np.pi
x = r * np.cos(theta)
y = r * np.sin(theta)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Draw unit circle boundary
circle = plt.Circle((0, 0), 1.0, fill=False, color='#667eea', linewidth=3, alpha=0.5)
ax.add_patch(circle)

# Plot points colored by hierarchy level
scatter = ax.scatter(x, y, c=levels, cmap='viridis', s=10, alpha=0.7)
cbar = plt.colorbar(scatter, ax=ax, label='Hierarchy Level')
cbar.ax.yaxis.label.set_color('white')
cbar.ax.tick_params(colors='white')

ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_aspect('equal')
ax.set_title(f'Poincaré Disk ({n_points_poincare} points, {n_levels} levels)', 
             fontsize=16, color='white', pad=20)
ax.set_xlabel('x', color='white', fontsize=12)
ax.set_ylabel('y', color='white', fontsize=12)
ax.grid(True, alpha=0.2)
ax.set_facecolor('#020306')

plt.tight_layout()
plt.show()

print(f"\n✅ Generated {n_points_poincare} points across {n_levels} hierarchy levels")
print(f"📊 Radial distribution:")
print(f"   Min radius: {r.min():.3f}")
print(f"   Max radius: {r.max():.3f}")
print(f"   Avg radius: {r.mean():.3f}")

## ⭐ Pareto Galaxy

Visualize non-dominated configurations in multi-objective space.

In [ ]:
#@title Generate Pareto Galaxy

n_configs = 300 #@param {type:"slider", min:100, max:1000, step:50}

# Generate random 5D configurations
np.random.seed(42)
configs = np.random.rand(n_configs, 5)

# Find non-dominated set (Pareto front)
def is_dominated(a, all_points):
    for b in all_points:
        if np.all(b <= a) and np.any(b < a):
            return True
    return False

print(f"Finding Pareto front from {n_configs} configurations...")
pareto_front = np.array([c for c in configs if not is_dominated(c, configs)])

# Project to 2D
weights_x = np.array([1.0, 0.2, 0.6, 0.5, 0.1])
weights_y = np.array([0.1, 1.0, 0.2, 0.6, 0.5])
weights_x /= np.linalg.norm(weights_x)
weights_y /= np.linalg.norm(weights_y)

all_x = configs @ weights_x
all_y = configs @ weights_y
pareto_x = pareto_front @ weights_x
pareto_y = pareto_front @ weights_y

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# All configurations
ax1.scatter(all_x, all_y, c=configs[:, 0], cmap='viridis', s=20, alpha=0.3, label='Dominated')
ax1.scatter(pareto_x, pareto_y, c=pareto_front[:, 0], cmap='plasma', 
            s=100, alpha=0.9, edgecolors='white', linewidths=2, label='Pareto Front', marker='*')
ax1.set_title(f'Multi-Objective Space ({len(pareto_front)}/{n_configs} non-dominated)', 
              fontsize=14, color='white')
ax1.set_xlabel('Objective 1 (projected)', color='white')
ax1.set_ylabel('Objective 2 (projected)', color='white')
ax1.legend(loc='upper right', fontsize=10)
ax1.grid(True, alpha=0.2)

# 3D view of Pareto front
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(pareto_front[:, 0], pareto_front[:, 1], pareto_front[:, 2],
            c=pareto_front[:, 0], cmap='plasma', s=100, alpha=0.8, edgecolors='white', linewidths=1)
ax2.set_title(f'Pareto Front (3D view)', fontsize=14, color='white')
ax2.set_xlabel('Accuracy', color='white')
ax2.set_ylabel('Latency', color='white')
ax2.set_zlabel('EPR-CV', color='white')
ax2.set_facecolor('#020306')
ax2.tick_params(colors='white')

plt.tight_layout()
plt.show()

print(f"\n✅ Found {len(pareto_front)} non-dominated configurations")
print(f"📊 Pareto ratio: {100*len(pareto_front)/n_configs:.1f}%")
print(f"\n📊 Best configurations (by objective 1):")
top_5 = np.argsort(pareto_front[:, 0])[:5]
for i, idx in enumerate(top_5, 1):
    print(f"  {i}. [{', '.join(f'{x:.3f}' for x in pareto_front[idx])}]")

## 🎬 Create Animation (Optional)

Generate an animated GIF showing topology evolution.

In [ ]:
#@title Generate Animation

n_frames = 30 #@param {type:"slider", min:10, max:60, step:5}
animation_mode = "barcode" #@param ["barcode", "landscape", "poincare"]

try:
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML
    
    print(f"Generating {n_frames}-frame animation of {animation_mode} mode...")
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    def update(frame):
        ax.clear()
        t = frame / n_frames * 2 * np.pi
        
        if animation_mode == "barcode":
            P = circle_clusters(n=400, k=3, noise=0.06 + 0.02*np.sin(t))
            result = ripser(P, maxdim=1)
            dgm1 = result['dgms'][1] if len(result['dgms']) > 1 else np.empty((0, 2))
            finite_dgm = dgm1[np.isfinite(dgm1[:, 1])]
            
            if len(finite_dgm) > 0:
                persistence = finite_dgm[:, 1] - finite_dgm[:, 0]
                order = np.argsort(-persistence)[:50]
                for i, idx in enumerate(order):
                    birth, death = finite_dgm[idx]
                    color = cm.viridis(persistence[idx] / persistence.max())
                    ax.plot([birth, death], [i, i], color=color, linewidth=2)
            
            ax.set_title(f'Barcode Evolution (frame {frame+1}/{n_frames})', color='white')
            ax.set_xlabel('Filtration parameter', color='white')
            ax.set_ylabel('Feature index', color='white')
        
        elif animation_mode == "landscape":
            P = swiss_roll(n=400)
            P = P + 0.5*np.sin(t)*np.array([[1, 0]])
            result = ripser(P, maxdim=0)
            dgm0 = result['dgms'][0]
            finite_dgm = dgm0[np.isfinite(dgm0[:, 1])]
            
            if len(finite_dgm) > 0:
                pl = PersistenceLandscape(dgms=[finite_dgm], homology_degree=0)
                xs = np.linspace(0, 3, 200)
                for k in range(1, 6):
                    ys = pl(xs, k)
                    if ys is not None:
                        ax.plot(xs, ys + 0.1*k, color=cm.viridis(k/6), linewidth=2)
            
            ax.set_title(f'Landscape Evolution (frame {frame+1}/{n_frames})', color='white')
            ax.set_xlabel('Parameter t', color='white')
            ax.set_ylabel('λₖ(t)', color='white')
        
        elif animation_mode == "poincare":
            levels = np.random.choice(6, size=400)
            r = 0.85 * (levels / 6)**0.9
            theta = np.random.rand(400) * 2 * np.pi + t
            x = r * np.cos(theta)
            y = r * np.sin(theta)
            
            circle = plt.Circle((0, 0), 1.0, fill=False, color='#667eea', linewidth=2)
            ax.add_patch(circle)
            ax.scatter(x, y, c=levels, cmap='viridis', s=10, alpha=0.7)
            ax.set_xlim(-1.1, 1.1)
            ax.set_ylim(-1.1, 1.1)
            ax.set_aspect('equal')
            ax.set_title(f'Poincaré Orbits (frame {frame+1}/{n_frames})', color='white')
        
        ax.grid(True, alpha=0.2)
        ax.set_facecolor('#020306')
    
    anim = FuncAnimation(fig, update, frames=n_frames, interval=100, repeat=True)
    plt.close()
    
    display(HTML(anim.to_jshtml()))
    print("\n✅ Animation complete! Use controls below to play/pause.")
    
except Exception as e:
    print(f"⚠️  Animation error: {e}")
    print("Try reducing n_frames or using a simpler mode.")

## 💾 Export Data

Save computed persistence diagrams and landscapes.

In [ ]:
# Save last computed persistence diagram
import pickle

if 'result' in locals():
    with open('persistence_diagrams.pkl', 'wb') as f:
        pickle.dump(result, f)
    print("✅ Saved persistence diagrams to persistence_diagrams.pkl")
    
    # Also save as text
    for i, dgm in enumerate(result['dgms']):
        np.savetxt(f'H{i}_diagram.txt', dgm, header=f'H{i} persistence diagram (birth, death)')
        print(f"✅ Saved H{i} diagram to H{i}_diagram.txt")
else:
    print("⚠️  No persistence data to save. Run a visualization first.")

## 📚 Resources

### Documentation
- [Ripser Documentation](https://ripser.scikit-tda.org/)
- [Persim Documentation](https://persim.scikit-tda.org/)
- [T-FAN GitHub](https://github.com/theadamsfamily1981-max/Quanta-meis-nib-cis)

### Papers
- Edelsbrunner & Harer (2010), *Computational Topology*
- Bubenik (2015), *Statistical Topological Data Analysis using Persistence Landscapes*
- Bauer et al. (2021), *Ripser: Efficient Computation of Vietoris-Rips Persistence Barcodes*

### More Notebooks
- [WebGL Version](https://colab.research.google.com/github/theadamsfamily1981-max/Quanta-meis-nib-cis/blob/main/notebooks/TFAN_Topology_Screensaver_WebGL.ipynb) - Interactive browser visualization
- [Full T-FAN Suite](https://github.com/theadamsfamily1981-max/Quanta-meis-nib-cis/tree/main/notebooks)

---

**Built with Ripser, Persim, and mathematical rigor** ✨